# Chapter 1: SMPC Example

| Chapter  | Colab   | Kaggle          | Gradient      | Studio Lab             | Binder             |
|:---------|:--------|:----------------|:--------------|:-----------------------|:-------------------|
| [Chapter 1: SMPC Example](1_privacy/Chapter_1_SMPC_Example.ipynb)                         | [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/matthew-mcateer/practicing_trustworthy_machine_learning/blob/main/1_privacy/Chapter_1_SMPC_Example.ipynb)       | [![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/matthew-mcateer/practicing_trustworthy_machine_learning/blob/main/1_privacy/Chapter_1_SMPC_Example.ipynb)       | [![Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/matthew-mcateer/practicing_trustworthy_machine_learning/blob/main/1_privacy/Chapter_1_SMPC_Example.ipynb)       | [![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/matthew-mcateer/practicing_trustworthy_machine_learning/blob/main/1_privacy/Chapter_1_SMPC_Example.ipynb)       | [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/matthew-mcateer/practicing_trustworthy_machine_learning/HEAD?urlpath=https%3A%2F%2Fgithub.com%2Fmatthew-mcateer%2Fpracticing_trustworthy_machine_learning%2Fblob%2Fmain%2F1_privacy%2FChapter_1_SMPC_Example.ipynb)              |

<!--
Originally found on GitHub at https://github.com/matthew-mcateer/practicing_trustworthy_machine_learning/blob/main/1_privacy/Chapter_1_SMPC_Example.ipynb
-->


The following is heavily based on an OpenMined tutorial written by [Ayoub Benaissa - Twitter: @y0uben11](https://twitter.com/y0uben11)

This tutorial has seen the most changes out of any in the book. **The code for the early release version of the book [can be found in the `deprecated` folder](https://github.com/matthew-mcateer/practicing_trustworthy_machine_learning/blob/main/1_privacy/deprecated/Chapter_1_SMPC_Example_DEPRECATED.ipynb), and is conceptually identical to the tutorial here, but thanks to the `HaGrid` package this version provides a much gentler introduct to steps such as uploading one's data before any computation takes place.** The original code is still valid with the package versions that have been specified, but Google Colab no longer supports those specific tutorials.

### A Note about dependencies

To install the OpenMined stack that you need in order to deploy a node, please run:

```bash
pip install -U syft hagrid --pre
```

PySyft is a library which contains the tools to run privacy preserving machine learning. Hagrid is a commandline tool that speeds up the deployment of PyGrid, the provider of a peer-to-peer network of data owners and data scientists who can collectively train AI model using Syft.

This tutorial is heavily reliant on docker.
As such, unlike the other notebooks for this book, it is unlikely to work in the above free GPU cloud resources without a lot of additional configuration (for example, here is a [Gist for installing Docker in Google Colab](https://gist.github.com/mwufi/6718b30761cd109f9aff04c5144eb885), which works as of November 2022, but is not guaranteed to work in the future.

In [ ]:
#@title PySyft & PyTorch Setup

# Install if not already using the provided conda environment
#!pip -qq install syft hagrid --pre
#!pip -qq install watermark
%load_ext watermark
%watermark -a "Practicing Trustworthy machine Learning" -u -d -v -m -p syft,loguru,torch,hagrid,crypten

### Launch the Doman Node

You only have one final step remaining now, before you unleash the power of Hagrid! The final step is to launch a domain node, which is as easy as:
```bash
hagrid launch <name_of_domain>
```

To stop the running domain, run:
```bash
hagrid land <name_of_domain>
```

But before stopping it, you can go to localhost:8081 in your browser to actually interact with the PyGrid Admin UI, where you can manage as a Data Owner your datasets, as well as incoming requests from data scientist. You can log in using the following credentials:

```text
info@openmined.org


changethis
```
Now you’re all set up to fully start using PySyft!

## Testing installation setup

In [2]:
import hagrid
from hagrid import wizard

In [3]:
# Check that Hagrid is working
wizard.check_hagrid

In [4]:
# Check that PySyft is installed and working
wizard.check_syft

In [6]:
# Check that Docker is installed
wizard.check_grid_docker

In [7]:
!hagrid launch alice_domain domain to docker:8081 --tag=latest --tail=false
# !hagrid land alice_domain --silent --force

✅ Updated HAGrid from branch: dev from branch: dev0m
⠙ Updating HAGrid from branch: dev
⠹ Checking for Docker Service   ice   
✅ Docker service is running
✅ Git 2.36.1
✅ Docker 20.10.20
✅ Docker Compose 2.12.1


                                             `
                                         `.+yys/.`
                                       ``/NMMMNNs`
                                    `./shNMMMMMMNs``    `..`
                                  `-smNMMNNMMMMMMN/.``......`
                                `.yNMMMMNmmmmNNMMm/.`....`
                              `:sdNMMMMMMNNNNddddds-`.`` `--. `
                           `.+dNNNNMMMMMMMMMNNNNmddohmh//hddy/.```..`
                          `-hNMMMMMMMMMMMMNNdmNNMNNdNNd:sdyoo+/++:..`
                        ../mMMMMMMMMMMMMMMNNmmmmNMNmNNmdmd/hNNNd+:`
                        `:mMNNMMMMMMMMMMMMNMNNmmmNNNNNdNNd/NMMMMm::
                       `:mMNNNMMMMMMMMMMMMMMMNNNNdNMNNmmNd:smMMmh//
                     ``/mMMMMMMMMMMMMMMMMMMMMMMNm

In [8]:
!hagrid launch bob_domain domain to docker:8082 --tag=latest --tail=false
# # !hagrid land bob_domain --silent --force

✅ Updated HAGrid from branch: dev from branch: dev0m
⠇ Updating HAGrid from branch: dev
⠏ Checking for Docker Service   ice   
✅ Docker service is running
✅ Git 2.36.1
✅ Docker 20.10.20
✅ Docker Compose 2.12.1


                                             `
                                         `.+yys/.`
                                       ``/NMMMNNs`
                                    `./shNMMMMMMNs``    `..`
                                  `-smNMMNNMMMMMMN/.``......`
                                `.yNMMMMNmmmmNNMMm/.`....`
                              `:sdNMMMMMMNNNNddddds-`.`` `--. `
                           `.+dNNNNMMMMMMMMMNNNNmddohmh//hddy/.```..`
                          `-hNMMMMMMMMMMMMNNdmNNMNNdNNd:sdyoo+/++:..`
                        ../mMMMMMMMMMMMMMMNNmmmmNMNmNNmdmd/hNNNd+:`
                        `:mMNNMMMMMMMMMMMMNMNNmmmNNNNNdNNd/NMMMMm::
                       `:mMNNNMMMMMMMMMMMMMMMNNNNdNMNNmmNd:smMMmh//
                     ``/mMMMMMMMMMMMMMMMMMMMMMMNm

In [9]:
# Check the Alice Domain
hagrid.check('localhost:8081',timeout=120)

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━┓
┃ PyGrid    ┃ Info                                      ┃    ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━┩
│ UI (βeta) │ http://localhost:8081/login               │ ✅ │
│ api       │ http://localhost:8081/api/v1/openapi.json │ ✅ │
└───────────┴───────────────────────────────────────────┴────┘

In [10]:
# Check the Bob Domain
hagrid.check('localhost:8082',timeout=120)

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━┓
┃ PyGrid    ┃ Info                                      ┃    ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━┩
│ UI (βeta) │ http://localhost:8082/login               │ ✅ │
│ api       │ http://localhost:8082/api/v1/openapi.json │ ✅ │
└───────────┴───────────────────────────────────────────┴────┘

In [11]:
import syft as sy
alice_domain = sy.login(
    email="info@openmined.org",
    password="changethis",
    port=8081
)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into alice_domain... done!

**Warning**: The syft version on your system and the node are different.
Version on your system: 0.7.0-beta.60
Version on the node: 0.7.0-beta.61



In [12]:
bob_domain = sy.login(
    email="info@openmined.org",
    password="changethis",
    port=8082
)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into bob_domain... done!

**Warning**: The syft version on your system and the node are different.
Version on your system: 0.7.0-beta.60
Version on the node: 0.7.0-beta.61



## Party 1 - Alice's team in the UK

In [ ]:
import syft as sy

sy.__version__

In [15]:
import syft as sy
import numpy as np
sy.logger.remove()

In [20]:
#@title Dataset creation

# run this cell
try:
   import pandas as pd
   alice_data = {'ID': ['011', '015'],
         'Age': [40, 39]}

   dataset = pd.DataFrame(alice_data)
   print(alice_dataset.head())
except Exception:
   print("Install the latest version of Pandas using the command: !pip install pandas")


In [23]:
["Rasswanth"] * training_data_alice.shape[0]

['Rasswanth', 'Rasswanth']

In [24]:
#@title Upload the dataset to Domain node

# run this cell
alice_data_subjects = sy.DataSubjectArray.from_objs(alice_dataset["ID"])

training_data_alice = sy.Tensor(
    alice_dataset["Age"]
).annotate_with_dp_metadata(
    lower_bound=0,
    upper_bound=100,
    data_subjects=alice_data_subjects
)

Tensor annotated with DP Metadata!
You can upload this Tensor to a domain node by calling `<domain_client>.load_dataset` and passing in this tensor as an asset.


In [26]:
# Upload a private dataset to the Domain object, as the root owner

# run this cell
alice_domain.load_dataset(
   assets={
      "Age_Data": training_data_alice,
   },
    name="Family_Age_Dataset",
   description="Our dataset contains the Ages of our four Family members with unique ID's. There are 2 columns and 4 rows in our dataset."
)

Loading dataset...
Loading dataset... checking assets...
Loading dataset... checking dataset name for uniqueness...
Loading dataset... checking dataset name for uniqueness...                                                                                                                    
Loading dataset... checking asset types...                              
Loading dataset... uploading...🚀                        

kj/filesystem-disk-unix.c++:1690: warning: PWD environment variable doesn't match current directory; pwd = /Users/matthewmcateer/Documents/GitHub/practicing_trustworthy_machine_learning
Uploading `Age_Data`: 100%|███████████████████████████████████████████| 1/1 [00:00<00:00,  4.31it/s]


Dataset is uploaded successfully !!! 🎉

Run `<your client variable>.datasets` to see your new dataset loaded into your machine!


In [27]:
alice_domain.datasets

Idx,Name,Description,Assets,Id
[0],Family_Age_Dataset,Our dataset contains the Ages of our four Family members with unique ID's. There are 2 columns and 4 rows in our dataset.,"[""Age_Data""] -> Tensor",3203340f-f271-4533-874e-c280352be12e


In [28]:
#@title Create a Data Scientist User

#change budget before demo
alice_domain.users.create(
    **{
        "name": "Seven of Nine",
        "email": "seven.ofnine@starfleet.com",
        "password": "borgcube",
        "budget":9_999_999
    }
)

User created successfully!


In [29]:
alice_domain.users


,id,email,name,budget,verify_key,role,added_by,website,institution,daa_pdf,created_at,budget_spent
0,1,info@openmined.org,Jane Doe,5.55,af29e51f2771bd7b911a29c9507bd4addcc16b7937aa64...,Owner,None,None,None,NaN,2022-12-05 09:27:52.832638,5.55
1,2,sheldon@caltech.edu,Sheldon Cooper,9999999.00,b9a85337dac82042a8ec7a7097fdd9b8f7d8b5753c330e...,Data Scientist,Jane Doe,,,1.0,2022-12-05 09:39:40.728098,9999999.00


In [30]:
#@title Accept/Deny Requests to the Domain
alice_domain.requests.pandas

""


In [48]:
#alice_domain.requests[-1].accept()

## Party 2 - Bob's team in Menlo Park

In [32]:
import syft as sy
import numpy as np
sy.logger.remove()

In [33]:
# Dataset Creation

# run this cell
try:
   import pandas as pd
   bob_data = {
       'ID': ['022', '034'],
       'Age': [9, 8]
   }

   bob_dataset = pd.DataFrame(bob_data)
   print(bob_dataset.head())
except Exception:
   print("Install the latest version of Pandas using the command: !pip install pandas")


    ID  Age
0  022    9
1  034    8


In [35]:
#@title Upload the dataset to Domain node

# run this cell
bob_data_subjects = sy.DataSubjectArray.from_objs(bob_dataset["ID"])

training_data_bob = sy.Tensor(
    bob_dataset["Age"]
).annotate_with_dp_metadata(
    lower_bound=0,
    upper_bound=100,
    data_subjects=bob_data_subjects
)


Tensor annotated with DP Metadata!
You can upload this Tensor to a domain node by calling `<domain_client>.load_dataset` and passing in this tensor as an asset.


In [36]:
#@title Upload a private dataset to the Domain object, as the root owner

# run this cell
bob_domain.load_dataset(
   assets={
      "age_data_train": training_data_bob,
      "age_data_test": training_data_bob,
   },
    name="Family_Age_Dataset",
   description="Our dataset contains the Ages of our four Family members with unique ID's. There are 2 columns and 4 rows in our dataset."
)


Loading dataset...
Loading dataset... checking assets...
Loading dataset... checking dataset name for uniqueness...
Loading dataset... checking dataset name for uniqueness...                                                                                                                    
Loading dataset... checking asset types...                              
Loading dataset... uploading...🚀                        

Uploading `Age_Data`: 100%|███████████████████████████████████████████| 1/1 [00:00<00:00,  4.64it/s]


Dataset is uploaded successfully !!! 🎉

Run `<your client variable>.datasets` to see your new dataset loaded into your machine!


In [37]:
bob_domain.datasets

Idx,Name,Description,Assets,Id
[0],Family_Age_Dataset,Our dataset contains the Ages of our four Family members with unique ID's. There are 2 columns and 4 rows in our dataset.,"[""Age_Data""] -> Tensor",07fff0f5-9fe7-47e9-b884-635857b5d43f


In [38]:
#@title Create a Data Scientist User

#change budget before demo
bob_domain.users.create(
    **{
        "name": "Seven of Nine",
        "email": "seven.ofnine@starfleet.com",
        "password": "borgcube",
        "budget":9_999_999
    }
)


User created successfully!


In [39]:
bob_domain.users

,id,email,name,budget,verify_key,role,added_by,website,institution,daa_pdf,created_at,budget_spent
0,1,info@openmined.org,Jane Doe,5.55,5c02b5692ee6ffe7bde572df3b7dc8cd14e3e44946f3c4...,Owner,None,None,None,NaN,2022-12-05 09:28:34.016374,5.55
1,2,sheldon@caltech.edu,Sheldon Cooper,9999999.00,110c611f62a527a44d70078fd8ddad472d8bcc5213d724...,Data Scientist,Jane Doe,,,1.0,2022-12-05 09:46:53.287861,9999999.00


In [40]:
#@title Accept/Deny Requests to the Domain
bob_domain.requests.pandas

""


In [49]:
!wget "https://raw.githubusercontent.com/facebookresearch/CrypTen/b1466440bde4db3e6e1fcb1740584d35a16eda9e/tutorials/mnist_utils.py" -O "mnist_utils.py"
     

--2022-12-05 01:55:06--  https://raw.githubusercontent.com/facebookresearch/CrypTen/b1466440bde4db3e6e1fcb1740584d35a16eda9e/tutorials/mnist_utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7401 (7.2K) [text/plain]
Saving to: 'mnist_utils.py'

mnist_utils.py      100%[===================>]   7.23K  --.-KB/s    in 0s      

2022-12-05 01:55:07 (17.3 MB/s) - 'mnist_utils.py' saved [7401/7401]



In [50]:

!python "mnist_utils.py" --option features --reduced 100 --binary
     

9913344it [00:00, 38114476.95it/s]                                              
Extracting /tmp/MNIST/raw/train-images-idx3-ubyte.gz to /tmp/MNIST/raw

29696it [00:00, 13955636.03it/s]                                                
Extracting /tmp/MNIST/raw/train-labels-idx1-ubyte.gz to /tmp/MNIST/raw

1649664it [00:00, 12395832.07it/s]                                              
Extracting /tmp/MNIST/raw/t10k-images-idx3-ubyte.gz to /tmp/MNIST/raw

5120it [00:00, 22070746.64it/s]                                                 
Extracting /tmp/MNIST/raw/t10k-labels-idx1-ubyte.gz to /tmp/MNIST/raw

Processing...
/Users/matthewmcateer/.asdf/installs/python/3.9.6/lib/python3.9/site-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make

# Data Scientist

Training an XOR network using PySyft
In this notebook we'll be training an XOR network using PySyft. What's special about this is that the training data will be divided between two domain nodes that are owned by different parties.

In [42]:
import syft as sy
import numpy as np
sy.logger.remove()

In [43]:
#@title Logging into the domain Nodes

# We will login into ALICE and BOB domain nodes
ALICE_node = sy.login(email="seven.ofnine@starfleet.com", password="borgcube", port=8081)
BOB_node = sy.login(email="seven.ofnine@starfleet.com", password="borgcube", port=8082)



Connecting to localhost... done! 	 Logging into alice_domain... done!

**Warning**: The syft version on your system and the node are different.
Version on your system: 0.7.0-beta.60
Version on the node: 0.7.0-beta.61

Connecting to localhost... done! 	 Logging into bob_domain... done!

**Warning**: The syft version on your system and the node are different.
Version on your system: 0.7.0-beta.60
Version on the node: 0.7.0-beta.61



In [44]:
ALICE_node.datasets


Idx,Name,Description,Assets,Id
[0],Family_Age_Dataset,Our dataset contains the Ages of our four Family members with unique ID's. There are 2 columns and 4 rows in our dataset.,"[""Age_Data""] -> Tensor",3203340f-f271-4533-874e-c280352be12e


In [45]:
BOB_node.datasets


Idx,Name,Description,Assets,Id
[0],Family_Age_Dataset,Our dataset contains the Ages of our four Family members with unique ID's. There are 2 columns and 4 rows in our dataset.,"[""Age_Data""] -> Tensor",07fff0f5-9fe7-47e9-b884-635857b5d43f


In [46]:
ALICE_node.privacy_budget

9999999.0

In [47]:
BOB_node.privacy_budget

9999999.0

In [ ]:
ALICE_node_train_data = ALICE_node.datasets[-1]["training_data"]
ALICE_node_targets_data = ALICE_node.datasets[-1]["training_targets"]
BOB_node_train_data = BOB_node.datasets[-1]["training_data"]
BOB_node_targets_data = BOB_node.datasets[-1]["training_targets"]


In [ ]:
train_data = ALICE_node_train_data.concatenate(BOB_node_train_data)
targets_data = ALICE_node_targets_data.concatenate(BOB_node_targets_data)
X = train_data
y = targets_data

In [ ]:
def relu(x,deriv=False):
    if deriv==True:
        return x>0
    return x*(x>0)

In [ ]:
layer0_weights = 2*np.random.random((3,4)) - 1
layer1_weights = 2*np.random.random((4,1)) - 1

In [ ]:
for j in range(1):
    # Forward propagation
    layer1_inputs = relu(X @ layer0_weights)  ; layer1_inputs.block
    layer2_inputs = relu(layer1_inputs @ layer1_weights) ; layer2_inputs.block 
    
    # Calculate errors
    layer2_inputs_delta = (y - layer2_inputs)* relu(layer2_inputs,deriv=True) ; layer2_inputs_delta.block
    layer1_inputs_delta = (layer2_inputs_delta@(layer1_weights.T)) * relu(layer1_inputs,deriv=True) ; layer1_inputs_delta.block
    
    # Update weights
    layer1_weights  = layer1_weights + layer1_inputs.T @ layer2_inputs_delta ; layer1_weights.block
    layer0_weights =  layer0_weights + X.T @ layer1_inputs_delta ; layer0_weights.block

In [ ]:
layer0_weights_dp = layer0_weights.publish(sigma=1e4)
layer1_weights_dp = layer1_weights.publish(sigma=1e4)

In [ ]:
print(layer0_weights_dp.get_copy())
print(layer1_weights_dp.get_copy())

Let's see how our privacy budget changed as a result of training for a single epoch:

In [ ]:
ALICE_node.privacy_budget

In [ ]:
BOB_node.privacy_budget

And voila! We've trained a neural network using PySyft's adversaril differential privacy system and its secure multiparty computation system working in tandem.